# Baseten custom pipeline use case

Let's assume that we want to upload the following pipeline to be served by baseten. Some differences with the standard use case are:
- It is a scikit-learn pipeline instead of a simple scikit-learn model
- Basides 3rd party libraries that could be install via a `requirements.txt` file, it has also local dependencies

In [2]:
import numpy as np
import sys
sys.path.insert(0, './pipelines/lr_model/')
sys.path.insert(0, './models/')
from joblib import dump
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from pipelines.lr_model.steps import (Disqualify, FeatureCalculator)

LEARNED_INTERCEPT = -1
LEARNED_COEFFICIENTS = [2, 1, 0, 0, 1]
lr_model = LogisticRegression()
lr_model.classes_ = np.array([0, 1])
lr_model.intercept_ = LEARNED_INTERCEPT
lr_model.coef_ = np.array([LEARNED_COEFFICIENTS])

pipeline = Pipeline(
    steps=[
        ("step1", Disqualify()),
        (
            "step2",
            FeatureCalculator(check_qualified=True),
        ),
        ("model", lr_model),
    ]
)

from bson.objectid import ObjectId

SID0 = ObjectId("abcdef123456abcdef123453")
SID1 = ObjectId("abcdef123456abcdef123453")
SID2 = ObjectId("abcdef123456abcdef123454")
SID3 = ObjectId("abcdef123456abcdef123454")
SID4 = ObjectId("abcdef123456abcdef543210")

BUILDER = {
    "feature1": SID0,
    "feature2": SID1,
    "feature3": SID4,
    "feature4": SID2,
    "feature5": SID1,
}

ROLE = {
    "feature1": SID0,
    "feature2": SID1,
    "feature3": SID1,
    "feature4": SID3,
    "feature5": SID4,
}

pipeline.predict_proba([(x, ROLE) for x in [BUILDER]])

dump(pipeline, "lr_model.joblib")

# somehow I should upload this pipeline and the necessary local dependencies to baseten

[({'feature1': ObjectId('abcdef123456abcdef123453'), 'feature2': ObjectId('abcdef123456abcdef123453'), 'feature3': ObjectId('abcdef123456abcdef543210'), 'feature4': ObjectId('abcdef123456abcdef123454'), 'feature5': ObjectId('abcdef123456abcdef123453')}, {'feature1': ObjectId('abcdef123456abcdef123453'), 'feature2': ObjectId('abcdef123456abcdef123453'), 'feature3': ObjectId('abcdef123456abcdef123453'), 'feature4': ObjectId('abcdef123456abcdef123454'), 'feature5': ObjectId('abcdef123456abcdef543210')})]


UFuncTypeError: Cannot cast ufunc 'expit' output from dtype('float64') to dtype('int64') with casting rule 'same_kind'